# Getting Started with Julia in Colab/Jupyter
You can either run this notebook in Google Colab, or using Jupyter on your own machine.

## Running on Google Colab
1. Work on a copy of this notebook: _File_ > _Save a copy in Drive_ (you will need a Google account). Alternatively, you can download the notebook using _File_ > _Download .ipynb_, then upload it to [Colab](https://colab.research.google.com/).
2. Execute the following cell (click on it and press Ctrl+Enter) to install Julia, IJulia (the Jupyter kernel for Julia) and other packages. You can update `JULIA_VERSION` and the other parameters, if you know what you're doing. Installation takes 2-3 minutes.
3. Reload this page (press Ctrl+R, or ⌘+R, or the F5 key) and continue to the _Checking the Installation_ section.

* _Note_: If your Colab Runtime gets reset (e.g., due to inactivity), repeat steps 2 and 3.

In [ ]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.6.0" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia DataFrames CSV Pipe"
JULIA_PACKAGES_IF_GPU="CUDA"
JULIA_NUM_THREADS=4
#---------------------------------------------------#

if [ -n "$COLAB_GPU" ] && [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  if [ "$COLAB_GPU" = "1" ]; then
      JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia  

  echo ''
  echo "Successfully installed `julia -v`!"
  echo "Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then"
  echo "jump to the 'Checking the Installation' section."
fi

Installing Julia 1.6.0 on the current Colab Runtime...
2022-01-23 11:53:01 URL:https://storage.googleapis.com/julialang2/bin/linux/x64/1.6/julia-1.6.0-linux-x86_64.tar.gz [112838927/112838927] -> "/tmp/julia.tar.gz" [1]
Installing Julia package IJulia...
Installing Julia package DataFrames...
Installing Julia package CSV...
Installing Julia package Pipe...
Installing IJulia kernel...
[ Info: Installing julia kernelspec in /root/.local/share/jupyter/kernels/julia-1.6

Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then
jump to the 'Checking the Installation' section.


## Checking the Installation
The `versioninfo()` function should print your Julia version and some other info about the system (if you ever ask for help or file an issue about Julia, you should always provide this information).

In [ ]:
versioninfo()

Julia Version 1.6.0
Commit f9720dc2eb (2021-03-24 12:55 UTC)
Platform Info:
  OS: Linux (x86_64-pc-linux-gnu)
  CPU: Intel(R) Xeon(R) CPU @ 2.20GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-11.0.1 (ORCJIT, broadwell)
Environment:
  JULIA_NUM_THREADS = 4


# Step 1. Import the necessary libraries

In [1]:
using DataFrames
using CSV
using Pipe
using Statistics

# Step 2. Import the dataset from this [address](https://raw.githubusercontent.com/justmarkham/DAT8/master/data/drinks.csv).


In [2]:
URL = "https://raw.githubusercontent.com/justmarkham/DAT8/master/data/drinks.csv"
download(URL,"drinks.csv")

"drinks.csv"

# Step 3. Assign it to a variable called drinks.


In [3]:
drinks = CSV.read("drinks.csv",DataFrame)

,country,beer_servings,spirit_servings,wine_servings,total_litres_of_pure_alcohol
,String31,Int64,Int64,Int64,Float64
1,Afghanistan,0,0,0,0.0
2,Albania,89,132,54,4.9
3,Algeria,25,0,14,0.7
4,Andorra,245,138,312,12.4
5,Angola,217,57,45,5.9
6,Antigua & Barbuda,102,128,45,4.9
7,Argentina,193,25,221,8.3
8,Armenia,21,179,11,3.8
9,Australia,261,72,212,10.4


# Step 4. Which continent drinks more beer on average?

In [4]:
continents = @pipe drinks |> groupby(_, :continent) |> combine(_, :beer_servings => mean)

,continent,beer_servings_mean
,String3,Float64
1,AS,37.0455
2,EU,193.778
3,AF,61.4717
4,NA,145.435
5,SA,175.083
6,OC,89.6875


# Step 5. For each continent print the statistics for wine consumption.

In [5]:
# Can't figure out a way now will get back to this at some point

# Step 6. Print the mean alcohol consumption per continent for every column

There has to be a better way to do this

In [6]:
continents = @pipe drinks |> groupby(_, :continent) |> combine(_, valuecols(_)[2:end] .=> mean)

,continent,beer_servings_mean,spirit_servings_mean,wine_servings_mean,total_litres_of_pure_alcohol_mean
,String3,Float64,Float64,Float64,Float64
1,AS,37.0455,60.8409,9.06818,2.17045
2,EU,193.778,132.556,142.222,8.61778
3,AF,61.4717,16.3396,16.2642,3.00755
4,NA,145.435,165.739,24.5217,5.99565
5,SA,175.083,114.75,62.4167,6.30833
6,OC,89.6875,58.4375,35.625,3.38125


In [14]:
@pipe drinks |> groupby(_, :continent) |> combine(_, names(_, Number) .=> mean)

,continent,beer_servings_mean,spirit_servings_mean,wine_servings_mean,total_litres_of_pure_alcohol_mean
,String3,Float64,Float64,Float64,Float64
1,AS,37.0455,60.8409,9.06818,2.17045
2,EU,193.778,132.556,142.222,8.61778
3,AF,61.4717,16.3396,16.2642,3.00755
4,NA,145.435,165.739,24.5217,5.99565
5,SA,175.083,114.75,62.4167,6.30833
6,OC,89.6875,58.4375,35.625,3.38125


# Step 7. Print the median alcohol consumption per continent for every column

In [7]:
@pipe drinks |> groupby(_, :continent) |> combine(_, valuecols(_)[2:end] .=> median)

,continent,beer_servings_median,spirit_servings_median,wine_servings_median,total_litres_of_pure_alcohol_median
,String3,Float64,Float64,Float64,Float64
1,AS,17.5,16.0,1.0,1.2
2,EU,219.0,122.0,128.0,10.0
3,AF,32.0,3.0,2.0,2.3
4,NA,143.0,137.0,11.0,6.3
5,SA,162.5,108.5,12.0,6.85
6,OC,52.5,37.0,8.5,1.75


In [13]:
@pipe drinks |> groupby(_,:continent) |> combine(_,names(_,Number) .=> median)

,continent,beer_servings_median,spirit_servings_median,wine_servings_median,total_litres_of_pure_alcohol_median
,String3,Float64,Float64,Float64,Float64
1,AS,17.5,16.0,1.0,1.2
2,EU,219.0,122.0,128.0,10.0
3,AF,32.0,3.0,2.0,2.3
4,NA,143.0,137.0,11.0,6.3
5,SA,162.5,108.5,12.0,6.85
6,OC,52.5,37.0,8.5,1.75


# Step 8. Print the mean, min and max values for spirit consumption.

In [22]:
compute_mean_max_min(x) = mean(x), minimum(x), maximum(x)
@pipe drinks |> groupby(_, :continent) |> combine(_, :spirit_servings => compute_mean_max_min)

,continent,spirit_servings_compute_mean_max_min
,String3,Tuple…
1,AS,"(60.8409, 0, 326)"
2,EU,"(132.556, 0, 373)"
3,AF,"(16.3396, 0, 152)"
4,NA,"(165.739, 68, 438)"
5,SA,"(114.75, 25, 302)"
6,OC,"(58.4375, 0, 254)"
